In [2]:
import requests
import json
import pandas as pd
import numpy as np
import time
from datetime import datetime
import os
from dotenv import load_dotenv

import mysql.connector

import gspread
from gspread.exceptions import APIError
from gspread.utils import rowcol_to_a1
#from gspread_formatting import *
#from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

import prettytable as pt

#from telegram import ParseMode
#from telegram.ext import CallbackContext, Updater

import logging

load_dotenv()

#asana creds
asana_token = os.getenv('ASANA_TOKEN')
workspace_gid = os.getenv('WORKSPACE_GID')
team_gid = os.getenv('TEAM_GID')

#telegram creds
bot_token = os.getenv('BOT_TOKEN')

#google sheets creds
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(creds)

#mysql creds
db_host = os.getenv("DB_HOST")
db_user = os.getenv("DB_USER")
db_pass = os.getenv("DB_PASS")
database = os.getenv("DATABASE")

#skip list
with open('users_to_skip.json', 'r', encoding='utf-8') as config_file:
    config = json.load(config_file)
users_to_skip = config.get('users_to_skip', '')
users_to_skip

['Artem Volovikov',
 'Smartics',
 'Andrey Komendant',
 'Татьяна Милова',
 'Артюшевский Дмитрий Игоревич',
 'Антонов Никита Сергеевич',
 'Дубов Константин Владимирович',
 'Евгений Подкорытов',
 'Анна Каменкова',
 'Антон Слукин',
 'Бухгалтер Арска',
 'Полина Кузнецова']

In [29]:
def get_tasks(access_token, user_gid, workspace_gid):
    
    # get user's mytasks list gid
    url = f"https://app.asana.com/api/1.0/users/{user_gid}/user_task_list"
    headers = {'Authorization': f'Bearer {access_token}'}
    
    payload = {
        'workspace': workspace_gid,
        'opt_fields': '',
        'opt_pretty': True  
        }
    
    response = requests.get(url, headers=headers, params=payload)
    status = response.status_code

    if status == 200:
        response_json = response.json()
    else:
        print(f'error: {status}')
    
    list_gid = response_json['data']['gid']
    
    # get my tasks for today
    
    my_tasks = []
    
    url = f"https://app.asana.com/api/1.0/user_task_lists/{list_gid}/tasks"
    
    payload = {
        'completed_since': 'now',
        'opt_fields': 'name, created_at, due_on, start_on, projects, projects.name, section.name, notes, assignee_section.name, created_by.name, created_by.gid, permalink_url',
        'limit': 100,
        'opt_pretty': True  
        }
    
     # pagination
    while True:
        response = requests.get(url, headers=headers, params=payload)
        
        if response.status_code == 200:
            json_data = response.json()
            
            if json_data.get('data'): 
                my_tasks.extend(json_data['data'])
            
            # check for more pages presence
            if json_data.get('next_page'): 
                payload['offset'] = json_data['next_page']['offset']  # update for next page
            else:
                break 
        else:
            print(f"error: {response.status_code}")
            break
        
    if my_tasks:
        my_tasks_df = pd.json_normalize(my_tasks, max_level=3) 
        my_tasks_df.rename(columns={'gid':'task_gid',
                                    'name':'task_name',
                                    'permalink_url':'url',
                                    'projects':'project_name'}, inplace=True)
    
        # SECTION NAME = TODAY и СЕГОДНЯ
        my_tasks_df = my_tasks_df[(my_tasks_df['assignee_section.name'] == 'Today') 
                                  | (my_tasks_df['assignee_section.name'] == 'Сегодня')]
        
        # extracting project names from nested list []
        if 'project_name' in my_tasks_df.columns:
            my_tasks_df['project_name'] = my_tasks_df['project_name'].apply(
                lambda x: x[0]['name'] if isinstance(x, list) and x else '')
        
        #re-order columns
        order = ['task_gid','project_name','task_name',
                    'start_on','due_on','notes',
                    'created_at','url','created_by.gid',
                    'created_by.name','assignee_section.gid','assignee_section.name']
        
        my_tasks_df = my_tasks_df[order]
        
    else:
        my_tasks_df = pd.DataFrame() 
        
    return my_tasks_df

access_token = '2/1200684451347406/1205697720563281:04f45ed138734b705ee6012155d06593'
user_gid = '1200684451347406'

df = get_tasks(access_token, user_gid, workspace_gid)
df

,task_gid,project_name,task_name,start_on,due_on,notes,created_at,url,created_by.gid,created_by.name,assignee_section.gid,assignee_section.name
0,1209032078309505,,Сделать MySQL table data partition через какое...,None,2025-01-31,,2024-12-23T07:15:07.003Z,https://app.asana.com/0/1209032078309505/12090...,1200684451347406,Elena Maeva,1200684497011158,Today
1,1208846925326940,,Проработать ф-л авторизации и основную структу...,None,2024-11-29,,2024-11-26T07:42:51.511Z,https://app.asana.com/0/1208846925326940/12088...,1200684451347406,Elena Maeva,1209030033487246,Сегодня


In [24]:
def get_list_gid(access_token, user_gid):

    url = f"https://app.asana.com/api/1.0/users/{user_gid}/user_task_list"
    headers = {'Authorization': f'Bearer {access_token}'}
    
    payload = {
        'workspace': workspace_gid,
        'opt_fields': '',
        'opt_pretty': True  
        }
    
    response = requests.get(url, headers=headers, params=payload)
    status = response.status_code

    if status == 200:
        response_json = response.json()
    else:
        print(f'error: {status}')
    
    list_gid = response_json['data']['gid']
    
    return list_gid

access_token = '2/1200684451347406/1205697720563281:04f45ed138734b705ee6012155d06593'
user_gid = '1200684451347406'

list_gid = get_list_gid(access_token, user_gid)
list_gid

'1200684497000735'

In [ ]:
conn = mysql.connector.connect(user = db_user,
                               password = db_pass,
                               host = db_host,
                               database = database)
cursor = conn.cursor()

        
cursor.execute("""
    INSERT INTO users (user_gid, user_name, user_token, list_gid)
    VALUES (%s, %s, %s, %s)
""", (
    user_gid,
    user_name,
    user_token,
    list_gid
))

conn.commit()
cursor.close()
conn.close()

In [30]:
import mysql.connector
import pandas as pd


conn = mysql.connector.connect(user=db_user,
                               password=db_pass,
                               host=db_host,
                               database=database)
cursor = conn.cursor()

insert_query = """INSERT INTO users (user_gid, name, user_token, list_gid) VALUES (%s, %s, %s, %s)"""

for idx, row in users_df_upd.iterrows():
    cursor.execute(insert_query, (row['user_gid'], row['name'], row['user_token'], row['list_gid']))

conn.commit()

cursor.close()
conn.close()

SyntaxError: invalid syntax (3058378189.py, line 13)

In [ ]:
conn = mysql.connector.connect(user = db_user,
                               password = db_pass,
                               host = db_host,
                               database = database)
cursor = conn.cursor()

insert_query = """INSERT INTO users (user_gid, name, user_token, list_gid) VALUES (%s, %s, %s, %s)"""
update_query = """UPDATE users SET name = %s, user_token = %s, list_gid = %s WHERE user_gid = %s"""

for index, row in users_df_upd.iterrows():

    cursor.execute("SELECT COUNT(*) FROM users WHERE user_gid = %s", (row['user_gid'],)) #check if user_gid exist
    exists = cursor.fetchone()[0]

    if exists:
        cursor.execute(update_query, (row['name'], row['user_token'], row['list_gid'], row['user_gid']))
    else:
        cursor.execute(insert_query, (row['user_gid'], row['name'], row['user_token'], row['list_gid']))

conn.commit()
cursor.close()
conn.close()